<a href="https://colab.research.google.com/github/reginaspatium/ab-test-performance-analysis/blob/main/A_B_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from google.colab import auth
from google.cloud import bigquery

In [ ]:
auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")

In [ ]:
query = """
WITH session_info AS(


SELECT
   sess.ga_session_id,
   sess.date,
   sespar.country,
   sespar.device,
   sespar.continent,
   sespar.channel,
   ab.test,
   ab.test_group
FROM `data-analytics-mate.DA.ab_test` AS ab
JOIN `DA.session` AS sess
ON ab.ga_session_id = sess.ga_session_id
JOIN `DA.session_params` AS sespar
ON sess.ga_session_id = sespar.ga_session_id
),


session_with_orders AS(


SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   COUNT(DISTINCT ord.ga_session_id) AS session_with_ord
FROM `DA.order` AS ord
JOIN session_info AS session_info
ON ord.ga_session_id = session_info.ga_session_id
GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group
),


events AS(


SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   evpar.event_name,
   COUNT(evpar.ga_session_id) AS event_cnt
FROM `DA.event_params` AS evpar
JOIN session_info
ON evpar.ga_session_id = session_info.ga_session_id
GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   evpar.event_name
),


session AS(


SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   COUNT(session_info.ga_session_id) AS session_cnt
FROM session_info
GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group
),


new_account AS (


SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   COUNT(DISTINCT acs.ga_session_id) AS new_account_cnt
FROM `DA.account_session` AS acs
JOIN session_info
ON acs.ga_session_id = session_info.ga_session_id
GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group
)


SELECT
   session_with_orders.date,
   session_with_orders.country,
   session_with_orders.device,
   session_with_orders.continent,
   session_with_orders.channel,
   session_with_orders.test,
   session_with_orders.test_group,
   'session with orders' AS event_name,
   session_with_orders.session_with_ord AS value
FROM session_with_orders


UNION ALL


SELECT
   events.date,
   events.country,
   events.device,
   events.continent,
   events.channel,
   events.test,
   events.test_group,
   event_name,
   event_cnt AS value
FROM events


UNION ALL


SELECT
   session.date,
   session.country,
   session.device,
   session.continent,
   session.channel,
   session.test,
   session.test_group,
   'session' AS event_name,
   session_cnt AS value
FROM session


UNION ALL


SELECT
   new_account.date,
   new_account.country,
   new_account.device,
   new_account.continent,
   new_account.channel,
   new_account.test,
   new_account.test_group,
   'new account' AS event_name,
   new_account_cnt AS value
FROM new_account
"""
query_job = client.query(query)
results = query_job.result()

df = results.to_dataframe()
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new account,1


порахуй статистичну значущість для чотирьох метрик:
add_payment_info / session
add_shipping_info / session
begin_checkout / session
new_accounts / session
Розраховуй значущість в тоталі по тесту, але щоб зробити свій проект унікальним та складнішим ти можеш порахувати результати в усіх можливих розрізах (в розрізі тестів, країн, пристроїв, тощо).

In [ ]:
# Нормалізація даних
df["event_name"] = df['event_name'].str.strip().str.lower().str.replace(" ", "_")
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new_account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new_account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new_account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new_account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new_account,1


In [ ]:
# Очищення метрик
metrics_for_test = ["add_payment_info", "add_shipping_info", "begin_checkout", "new_account", "session"]
df_filtered = df[df['event_name'].isin(metrics_for_test)]

# Pivot потрібних метрик
pivot_table_metrics = pd.pivot_table(
    data = df_filtered,
    values= "value",
    index = ["test", "test_group", "continent", "device"],
    columns = "event_name",
    aggfunc = "sum",
    fill_value=0
).reset_index()

display(pivot_table_metrics)

event_name,test,test_group,continent,device,add_payment_info,add_shipping_info,begin_checkout,new_account,session
0,1,1,(not set),desktop,0,1,1,6,55
1,1,1,(not set),mobile,7,3,3,0,39
2,1,1,(not set),tablet,0,0,0,1,3
3,1,1,Africa,desktop,12,16,20,18,285
4,1,1,Africa,mobile,7,25,34,16,198
...,...,...,...,...,...,...,...,...,...
139,4,2,Europe,mobile,259,359,891,630,7606
140,4,2,Europe,tablet,13,27,68,49,478
141,4,2,Oceania,desktop,21,39,89,51,629
142,4,2,Oceania,mobile,17,20,48,36,408


**Оскільки проводиться аналіз конверсії на великих вибірках, використовується Z-тест для пропорцій, який не потребує перевірки на нормальність розподілу.**

In [ ]:
# Цільові метрики та розрізи для аналізу
metrics = ["add_payment_info", "add_shipping_info", "begin_checkout", "new_account"]
continents = pivot_table_metrics["continent"].unique()
device = pivot_table_metrics["device"].unique()

# Порожній список для збору результатів
ab_results = []

# РОЗРАХУНОК ЗАГАЛЬНИХ РЕЗУЛЬТАТІВ
for test_id in pivot_table_metrics['test'].unique():
    current_test = pivot_table_metrics[pivot_table_metrics['test'] == test_id]
    # Сумування показників по групах
    group_A_total = current_test[current_test["test_group"] == 1].sum(numeric_only=True)
    group_B_total = current_test[current_test["test_group"] == 2].sum(numeric_only=True)

    # Розрахунок чисельників (успішні дії) та знаменників (сесії)
    for metric in metrics:
        num_c, num_t = group_A_total[metric], group_B_total[metric]
        den_c, den_t = group_A_total["session"], group_B_total["session"]

        # Умова для розрахунку за наявності даних в обох групах
        if den_c > 0 and den_t > 0:
          # Z-тест для перевірки статистичної значущості
            z_stat, p_value = proportions_ztest(count=[num_t, num_c], nobs=[den_t, den_c])

            # Отримання загального результату - "All Regions"
            ab_results.append({
                'test_number': test_id,
                'continent': 'All Regions',
                'device': 'All Devices',
                'metric': metric,
                'numerator_test': num_t,
                'denominator_test': den_t,
                'conversion_test': num_t / den_t,
                'numerator_control': num_c,
                'denominator_control': den_c,
                'conversion_control': num_c / den_c,
                'metric_change_pct': round(((num_t/den_t)/(num_c/den_c)-1)*100, 2),
                'p_value': round(p_value, 4),
                'z_score': round(z_stat, 4),
                'significant': p_value < 0.05
            })

# РОЗРАХУНОК ДЕТАЛІЗОВАНИХ РЕЗУЛЬТАТІВ
# Створення циклів по кожному тесту, континенту та типу пристрою
for test_id in pivot_table_metrics['test'].unique():
    for cont in continents:
        for dev in device:
          # Фільтрація таблиці за сегментами
            current_slice = pivot_table_metrics[
                (pivot_table_metrics['test'] == test_id) &
                (pivot_table_metrics['continent'] == cont) &
                (pivot_table_metrics['device'] == dev)
            ]

            # Розподіл даних на контрольну (gA) та тестову (gB) групи
            gA, gB = current_slice[current_slice["test_group"] == 1], current_slice[current_slice["test_group"] == 2]

            # Перевірка, чи є дані в обох групах
            if not gA.empty and not gB.empty:
                for metric in metrics:
                    num_c, num_t = gA[metric].iloc[0], gB[metric].iloc[0]
                    den_c, den_t = gA["session"].iloc[0], gB["session"].iloc[0]

                    # Розрахунок стат. тесту
                    if den_c > 0 and den_t > 0:
                        z_stat, p_value = proportions_ztest(count=[num_t, num_c], nobs=[den_t, den_c])
                        ab_results.append({
                            'test_number': test_id,
                            'continent': cont,
                            'device': dev,
                            'metric': metric,
                            'numerator_test': num_t,
                            'denominator_test': den_t,
                            'conversion_test': num_t / den_t,
                            'numerator_control': num_c,
                            'denominator_control': den_c,
                            'conversion_control': num_c / den_c,
                            'metric_change_pct': round(((num_t/den_t)/(num_c/den_c)-1)*100, 2),
                            'p_value': round(p_value, 4),
                            'z_score': round(z_stat, 4),
                            'significant': p_value < 0.05
                        })

# Конвертація у DataFrame
final_df = pd.DataFrame(ab_results)

# Обробка пропущених назв регіонів
final_df['continent'] = final_df['continent'].replace('(not set)', 'Unknown')

# Експорт у формат CSV
final_df.to_csv('ab_test_final_master.csv', index=False)

display(final_df.head())

/tmp/ipython-input-2874794514.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  'metric_change_pct': round(((num_t/den_t)/(num_c/den_c)-1)*100, 2),
/tmp/ipython-input-2874794514.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  'metric_change_pct': round(((num_t/den_t)/(num_c/den_c)-1)*100, 2),
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std
/tmp/ipython-input-2874794514.py:79: RuntimeWarning: invalid value encountered in scalar divide
  'metric_change_pct': round(((num_t/den_t)/(num_c/den_c)-1)*100, 2),
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std
/tmp/ipython-input-2874794514.py:79: RuntimeWarning: invalid value encountered in scalar divide
  'metric_change_pct': round(((num_t/den_t)/(num_c/den_c)-1)*100, 2),
/usr/local/lib/p

,test_number,continent,device,metric,numerator_test,denominator_test,conversion_test,numerator_control,denominator_control,conversion_control,metric_change_pct,p_value,z_score,significant
0,1,All Regions,All Devices,add_payment_info,2229,45193,0.049322,1988,45362,0.043825,12.54,0.0001,3.9249,True
1,1,All Regions,All Devices,add_shipping_info,3221,45193,0.071272,3034,45362,0.066884,6.56,0.0092,2.6036,True
2,1,All Regions,All Devices,begin_checkout,4021,45193,0.088974,3784,45362,0.083418,6.66,0.0029,2.9788,True
3,1,All Regions,All Devices,new_account,3681,45193,0.081451,3823,45362,0.084278,-3.35,0.1229,-1.5429,False
4,2,All Regions,All Devices,add_payment_info,2409,50244,0.047946,2344,50637,0.046290,3.58,0.2146,1.2410,False


##**Аналіз результатів А/В тестування**

###**Етапи проєкту**

**Вивантаження та агрегація даних (SQL):**
*   Використання BigQuery для об'єднання таблиць сесій, параметрів та подій.
*   Розрахунок кількості унікальних подій (event_cnt, session_with_ord, new_account_cnt) у розрізі дат, країн, пристроїв та каналів.

**Підготовка та очищення даних (Python):**

*   Нормалізація назв метрик, фільтрація цільових показників та трансформація даних за допомогою pivot_table для підготовки до статистичного тестування

**Сегментація:** Аналіз загальних результатів та у розрізі континентів та типів пристроїв.

**Статистичне тестування:** Застосування Z-тесту для пропорцій (proportions_ztest) для оцінки значущості різниці в конверсіях між контрольною (Group А) та тестовою (Group В) групами.

**Автоматизація висновків:** Програмне обчислення Z-score, P-value та автоматичне визначення статусу значущості (significant).

**Візуалізація:** Побудова інтерактивного дашборду в Tableau для бізнес-користувачів.

###**Статистична методологія (pеалізація на Python)**
Для перевірки гіпотез у коді використано бібліотеку **statsmodels**.

**Реалізація:** Скрипт автоматично ітерує через усі комбінації континентів та пристроїв, що дозволяє виявити, наприклад, чи був успіх тесту рівномірним для Mobile та Desktop.3.

##**Фінальні висновки**


**Ключовий драйвер росту:** Впровадження змін у Test 1 призвело до статистично підтвердженого зростання конверсії в етап додавання платіжних даних (add_payment_info).

Z-score = 3.9249 (P-value < 0.001) свідчить про дуже **високу надійність результату.**

Відносний приріст склав +12.54%.

**Також зафіксовано значуще покращення на етапах:**
*   **begin_checkout**: приріст +6.66% (Z = 2.9788).
*   **add_shipping_info**: приріст +6.56% (Z = 2.6036).

Метрика **new_account** показала від'ємну динаміку (-3.35%), проте Z-score = -1.5429, що означає, що це падіння є випадковим і не пов'язаним зі змінами в тесті.

**Рекомендація:** Оскільки тест позитивно вплинув на ключові конверсійні кроки без статистично значущої шкоди для реєстрацій, рекомендується впровадити тестове рішення (Group В) на 100% користувачів.


###**Корисні посилання:**

[Інтерактивний дашборд Tableau](https://public.tableau.com/app/profile/regina.dotsenko/viz/A_Btests/ABTestingTool?publish=yes)

[Фінальний набір даних (CSV)](https://drive.google.com/file/d/1mjEHdFhkyR4ZICcwTAlulYrjqshVp7vn/view?usp=sharing)
